In [ ]:
# basic
import os 
import numpy as np
import xarray as xr
import geopandas as gpd

# climate-related
import cftime
import regionmask
from xclim import sdba
from datetime import datetime

import warnings
warnings.simplefilter("ignore") 

chunks_dict = {"lon": 10, "lat": 10, "time": -1}

In [ ]:
# prepare baseline climate for bias correction of projections 
pp_baseline  = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc")["prcp"]
t2m_baseline = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc")["temp"]
pp_baseline

In [ ]:
# future climate tasks
period = slice("1980-01-01", "2099-12-31") # period
days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

os.chdir("/home/rooda/Dropbox/Patagonia/Data/Future")
gcm_list  = ["ACCESS-CM2", "EC-Earth3", "INM-CM5-0", "KACE-1-0-G", "MPI-ESM1-2-HR", 
             "CMCC-ESM2",  "GFDL-ESM4",  "IPSL-CM6A",  "MIROC6",   "NorESM2-LM"]
ssp_list  = ["ssp245","ssp370","ssp585"] 
start = datetime.now()

for gcm in gcm_list:
    pp_historical  = xr.open_mfdataset(os.path.join(gcm + "/*_historical_*.nc"))["pr"]
    t2m_historical = xr.open_mfdataset(os.path.join(gcm + "/*_historical_*.nc"))[["tasmax","tasmin"]]
    
    for ssp in ssp_list:
        path = os.path.join(gcm + "/*" + ssp + "*.nc")
        pp_model_ssp = xr.open_mfdataset(path)["pr"]
        pp_model_ssp = xr.concat([pp_historical, pp_model_ssp], dim = "time").drop("height")
        pp_model_ssp.coords['lon'] = (pp_model_ssp.coords['lon'] + 180) % 360 - 180
        pp_model_ssp = pp_model_ssp.sortby(pp_model_ssp.lon)
        pp_model_ssp = pp_model_ssp.resample(time='MS').sum()
        pp_model_ssp = pp_model_ssp*86400
        pp_model_ssp = pp_model_ssp.where((pp_model_ssp.lon >= -76) & (pp_model_ssp.lon <= -68), drop=True)
        pp_model_ssp = pp_model_ssp.sel(time=~((pp_model_ssp.time.dt.month == 2) & (pp_model_ssp.time.dt.day == 29)))
        if isinstance(pp_model_ssp.indexes['time'][0], cftime.DatetimeNoLeap) or isinstance(pp_model_ssp.indexes['time'][0], cftime.Datetime360Day):
            pp_model_ssp["time"] = pp_model_ssp.indexes['time'].to_datetimeindex()
        pp_model_ssp = pp_model_ssp.chunk(chunks_dict)
        pp_model_ssp.attrs["units"] = "mm"
        pp_model_ssp  = pp_model_ssp.sel(time=period)
        pp_model_ssp  = pp_model_ssp.astype("int16")
        
        t2m_model_ssp = xr.open_mfdataset(path)[["tasmax","tasmin"]]
        t2m_model_ssp = xr.concat([t2m_historical, t2m_model_ssp], dim = "time").drop("height")
        t2m_model_ssp = t2m_model_ssp.resample(time='MS').mean()
        t2m_model_ssp["tas"] = (t2m_model_ssp.tasmax + t2m_model_ssp.tasmin)/2
        t2m_model_ssp = t2m_model_ssp["tas"] 
        t2m_model_ssp.coords['lon'] = (t2m_model_ssp.coords['lon'] + 180) % 360 - 180
        t2m_model_ssp = t2m_model_ssp.sortby(t2m_model_ssp.lon)
        t2m_model_ssp = t2m_model_ssp-273.15
        t2m_model_ssp = t2m_model_ssp.where((t2m_model_ssp.lon >= -76) & (t2m_model_ssp.lon <= -68), drop=True)
        t2m_model_ssp = t2m_model_ssp.sel(time=~((t2m_model_ssp.time.dt.month == 2) & (t2m_model_ssp.time.dt.day == 29)))
        if isinstance(t2m_model_ssp.indexes['time'][0], cftime.DatetimeNoLeap) or isinstance(t2m_model_ssp.indexes['time'][0], cftime.Datetime360Day):
            t2m_model_ssp["time"] = t2m_model_ssp.indexes['time'].to_datetimeindex()
        t2m_model_ssp = t2m_model_ssp.chunk(chunks_dict)
        t2m_model_ssp.attrs["units"] = "degC"
        t2m_model_ssp  = t2m_model_ssp.sel(time=period)
        
        t2m_model_ssp.to_netcdf("/home/rooda/OGGM_results/Future_climate/T2M_" + gcm + "_" + ssp + ".nc")
        pp_model_ssp.to_netcdf("/home/rooda/OGGM_results/Future_climate/PP_"   + gcm + "_" + ssp + ".nc")
        print(gcm, ssp, datetime.now()-start)